# Batch ETL process with PySpark

The notebook's purpose is to do a batch processing for csv files from the retail company, also is presented the EDA process and store it on postgres cloud service

In [1]:
!pip install pyspark --break-system-packages
!pip install -q findspark --break-system-packages

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import findspark
from pyspark.sql import SparkSession
import pyspark
from pyspark.sql.functions import col, max, count, when, udf, to_date, sum, greatest, lit, coalesce, upper
from pyspark.sql.types import TimestampType

from pandas import to_datetime, NaT

from dotenv import load_dotenv
import os
import time, random

In this section we initialize the pyspark session and configure the session to use the jar component to connect with postgresql 

In [3]:
findspark.init()
spark = (SparkSession.builder.master('local[*]')
         .appName('Batch processing')
         .config('spark.jars', '../../libraries/postgresql-42.7.5.jar')
         .getOrCreate()
)

25/04/08 17:51:25 WARN Utils: Your hostname, alexrm resolves to a loopback address: 127.0.1.1; using 192.168.100.74 instead (on interface wlp0s20f3)
25/04/08 17:51:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/04/08 17:51:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Reading data

Reading csv files stored in the data folder: `products.csv`, `sales.csv` and `soh.csv`

In [4]:
products = (
    spark.read.format('csv')
    .option('header', 'true')
    .option('inferSchema', 'true')
    .load('../../data/products.csv')
)
products.printSchema()

root
 |-- gtin: long (nullable = true)
 |-- productCode: string (nullable = true)
 |-- size: string (nullable = true)
 |-- color: string (nullable = true)
 |-- label: string (nullable = true)
 |-- category: string (nullable = true)



In [5]:
sales = (
    spark.read.format('csv')
    .option('header', 'true')
    .option('inferSchema', 'true')
    .load('../../data/sales*.csv')
)
sales.printSchema()

[Stage 3:============================================>              (6 + 2) / 8]

root
 |-- sku: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- site_code: string (nullable = true)
 |-- date: string (nullable = true)



In [6]:
soh = (
    spark.read.format('csv')
    .option('header', 'true')
    .option('inferSchema', 'true')
    .load('../../data/soh.csv')
)
soh.printSchema()

25/04/08 17:51:44 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
                                                                                

root
 |-- site_code: string (nullable = true)
 |-- sku: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- date: string (nullable = true)



### EDA ANALYSIS

The section must be visible as EDA analysis and also do some transformations to the data, erasing rows to only include `clean` data into the cloud database

In [7]:
products.describe().show()

25/04/08 17:51:47 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+--------------------+-----------+------------------+------+-------------+-----------+
|summary|                gtin|productCode|              size| color|        label|   category|
+-------+--------------------+-----------+------------------+------+-------------+-----------+
|  count|                 451|        451|               451|   451|          451|        451|
|   mean|9.780123459034512E12|       NULL|14.813333333333333|  NULL|         NULL|       NULL|
| stddev|  1303.3406465853573|       NULL|10.991020085620919|  NULL|         NULL|       NULL|
|    min|       9780123456789| A-LINE-044|                10| Beige| A Line Shift|Accessories|
|    max|       9780123461288|   YOGA-423|                XS|Yellow|Zip Up Hoodie|       Tops|
+-------+--------------------+-----------+------------------+------+-------------+-----------+



In [8]:
sales.describe().show()

[Stage 9:====================================>                      (5 + 3) / 8]

+-------+----------+-----------------+---------+------------------+
|summary|       sku|         quantity|site_code|              date|
+-------+----------+-----------------+---------+------------------+
|  count|   1900538|          1900368|  1900262|           1899906|
|   mean|      NULL|6.396592659947968|     NULL|              NULL|
| stddev|      NULL|6.464789205766256|     NULL|              NULL|
|    min|A-LINE-044|              -15|   AUS000|        01-01-2023|
|    max|  YOGA-423|               15|   usa004|September 30, 2024|
+-------+----------+-----------------+---------+------------------+



In [9]:
soh.describe().show()

[Stage 12:==================================================>       (7 + 1) / 8]

+-------+---------+----------+-----------------+------------------+
|summary|site_code|       sku|         quantity|              date|
+-------+---------+----------+-----------------+------------------+
|  count|  8619047|   8619977|          8619162|           8620285|
|   mean|     NULL|      NULL|574.3629137032116|              NULL|
| stddev|     NULL|      NULL|568.1851194776879|              NULL|
|    min|   AUS000|A-LINE-044|            -1495|        01-01-2023|
|    max|   usa004|  YOGA-423|             1496|September 30, 2024|
+-------+---------+----------+-----------------+------------------+



In [10]:
products = products.withColumnRenamed('productCode', 'product_code')

In [11]:
def count_non_null(df):
    """
    Count non null rows based on each column of the Pyspark DataFrame and print in the console

    Parameter: 
    df (pyspark.DataFrame)
    """
    df.select([count(when(col(c).isNotNull(), c)).alias(c) for c in df.columns]).show()

In [12]:
def extract_outliers(df, column):
    quartiles = df.approxQuantile(column, [0.25, 0.75], 0.01)
    Q1, Q3 = quartiles[0], quartiles[1]

    IQR = Q3 - Q1

    lower_bound, upper_bound = Q1 - 1.5*IQR, Q3 + 1.5*IQR
    clean = df.filter((df[column] >= lower_bound) & (df[column] <= upper_bound))
    print(f'Qty of rows erased: {((clean.count() / df.count())*100):2f}') 
    return clean

In [13]:
count_non_null(products)

+----+------------+----+-----+-----+--------+
|gtin|product_code|size|color|label|category|
+----+------------+----+-----+-----+--------+
| 451|         451| 451|  451|  451|     451|
+----+------------+----+-----+-----+--------+



In [14]:
count_non_null(sales)

[Stage 18:=============================>                            (4 + 4) / 8]

+-------+--------+---------+-------+
|    sku|quantity|site_code|   date|
+-------+--------+---------+-------+
|1900538| 1900368|  1900262|1899906|
+-------+--------+---------+-------+



In [15]:
count_non_null(soh)

[Stage 21:=======>                                                  (1 + 7) / 8]

+---------+-------+--------+-------+
|site_code|    sku|quantity|   date|
+---------+-------+--------+-------+
|  8619047|8619977| 8619162|8620285|
+---------+-------+--------+-------+



In [16]:
extract_outliers(sales, 'quantity')

Qty of rows erased: 93.583164


DataFrame[sku: string, quantity: int, site_code: string, date: string]

In [17]:
extract_outliers(soh, 'quantity')

[Stage 37:>                                                         (0 + 8) / 8]

Qty of rows erased: 93.373159


DataFrame[site_code: string, sku: string, quantity: int, date: string]

In [18]:
def parse_date(date):
    """
    Parse a date which could be in different forms

    Parameters:
    date (str): A date in a string datatype

    Return:
    (datetime) Date parsed to the format: yyyy-mm-dd
    """
    date = to_datetime(date, format='mixed', errors='coerce')
    if date is NaT:
        return None
    return date


In [19]:
def sample(df, fraction=0.1):
    """
    Select randomly a fraction of rows from a dataframe

    Parameters:
    df (pyspark.DataFrame): dataframe
    fraction (double): The portion of data that should be added 
    """
    return df.sample(withReplacement=False, fraction=fraction, seed=time.time()).show()

In [20]:
def drop_null_values(df):
    """
    Drop null values of a dataframe on any column and also get a statistics before droping the values

    Parameters:
    df (pyspark.DataFrame)

    Return: 
    (pyspark.DataFrame) return a new dataframe erasing the rows with null values
    """
    df_clean = df.dropna(how='any', subset=None)
    initial_shape, clean_shape = df.count(), df_clean.count()
    print(f'Percentage of data that will be erased: {(100-clean_shape/initial_shape*100.0):.2f}')
    return df_clean

In [21]:
drop_null_values(sales)
drop_null_values(soh)

Percentage of data that will be erased: 10.04


[Stage 49:=======>                                                  (1 + 7) / 8]

Percentage of data that will be erased: 10.00


DataFrame[site_code: string, sku: string, quantity: int, date: string]

In [22]:
def pipeline(df, functions):
    for f in functions:
        df = f(df)
    return df

defining a UDF (User Definition Function) to apply the function `parse_date` to pyspark.DataFrames

In [23]:
parse_date_udf = udf(parse_date, TimestampType())

In this specific case we could see that if we intend to drop null values from dataframes: 
- 'soh' -> have a small portion of the data length to erase, so it could be safe and add a small variation in the data
- 'sales' -> have a small portion but we have to decide if erase or try to fix some values in each column.

In [24]:
soh = pipeline(
    soh, 
    [
        lambda df: df.fillna({'quantity': 0}), 
        lambda df: df.select(
            col('sku'),
            col('date'),
            upper(col('site_code')).alias('site_code'),
            greatest(col('quantity'), lit(0)).alias('quantity')
        ),
        drop_null_values,
        lambda df: extract_outliers(df, 'quantity'),
        lambda df: df.withColumn('date', to_date(parse_date_udf(col('date')))), # parse dates
    ]
)

Percentage of data that will be erased: 7.50


[Stage 63:=======>                                                  (1 + 7) / 8]

Qty of rows erased: 100.000000


In the case of `sales` dataframe, its obvious to think that if there are sales then the quantity must be a value more than 0. Also doing more research on this retail company working with clothes, we see that there are not fractional products, so for this reasons, we see that a sales quantity must be greater than 0 and also quantity is in the natural numbers set. So the conclusion is that quantity must be at least 1 to be a valid record.

In [25]:
sales = pipeline(
    sales,
    [        
        lambda df: df.fillna({'quantity': 1}),
        lambda df: df.select(
            col('sku'),
            col('date'),
            upper(col('site_code')).alias('site_code'),
            greatest(col('quantity'), lit(1)).alias('quantity')
        ),
        drop_null_values,
        #lambda df: extract_outliers(df, 'quantity'),
        lambda df: df.withColumn('date', to_date(parse_date_udf(col('date'))))
    ]
)

[Stage 69:=============================>                            (4 + 4) / 8]

Percentage of data that will be erased: 7.53


In [26]:
products = pipeline(
    products,
    [
        lambda df: df.withColumnRenamed('productCode', 'product_code'),
    ]
)

In [27]:
count_non_null(sales)
sample(sales, 0.01)
#sales.count()

+-------+-------+---------+--------+
|    sku|   date|site_code|quantity|
+-------+-------+---------+--------+
|1802336|1802336|  1802336| 1802336|
+-------+-------+---------+--------+



1802336

In [28]:
count_non_null(soh)
sample(soh, 0.0001)

ERROR:root:KeyboardInterrupt while sending command.                 (0 + 8) / 8]
Traceback (most recent call last):
  File "/home/alex/.local/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/alex/.local/lib/python3.12/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/socket.py", line 707, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
count_non_null(products)
sample(products, 0.01)

In [ ]:
soh = (
    soh
    .withColumn('date', to_date(parse_date_udf(col('date'))))
    .groupby('site_code', 'sku', 'date')
    .agg(sum('quantity').alias('quantity'))
)

### Saving in cloud storage

In this last section the cleaned data is stored into a cloud database based on postgres.

In [ ]:
jdbc_url = os.getenv('DB_JDBC_URL')
properties = {
    "user": os.getenv('DB_USER'),
    "password": os.getenv('DB_PASSWORD'),
    "driver": "org.postgresql.Driver"
}

In [ ]:
products.write.jdbc(url=jdbc_url, table='products', mode='append', properties=properties)

In [ ]:
sales.write.jdbc(url=jdbc_url, table='sales', mode='append', properties=properties)

In [ ]:
soh.write.jdbc(url=jdbc_url, table='soh', mode='append', properties=properties)